In [1]:
!pip install pyaspeller
!pip install deep_translator 
!pip install pymorphy2

In [2]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import itertools
from scipy import sparse
%matplotlib inline

import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

from multiprocessing import Pool
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

from textblob import TextBlob
from pyaspeller import YandexSpeller
from deep_translator import GoogleTranslator

from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate

import pickle


np.random.seed(42)
random.seed(42)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrakozevnikova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandrakozevnikova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train_df = pd.read_csv('train_ml.csv')

In [4]:
test_df = pd.read_csv('new_test_ml.csv')

In [11]:
train_df.head(2)

,bank,feeds,grades,date
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,16.02.2017 16:10
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,13.12.2016 1:05


In [12]:
test_df.head(2)

,Unnamed: 0,bank,feeds,date
0,0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53
1,1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19


## **Стемминг**

In [13]:
def stemming_texts(texts):
    st = SnowballStemmer("russian")
    stem_text = []
    for text in tqdm(texts):
        word_tokens = word_tokenize(text)
        stem_text.append(' '.join([st.stem(word) for word in word_tokens]))
    return stem_text

In [16]:
train_df['stem_text'] = stemming_texts(train_df.prep_feeds.tolist())

AttributeError: 'DataFrame' object has no attribute 'prep_feeds'

In [19]:
test_df['stem_text'] = stemming_texts(test_df.prep_feeds.tolist())

100%|██████████| 17220/17220 [01:54<00:00, 149.98it/s]


In [20]:
#train_df.to_csv('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/stem_prep_train.csv', index=False)
#test_df.to_csv('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/stem_prep_test.csv', index=False)

# Работаем дальше

In [62]:
train_pass = '/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/stem_prep_train.csv'
test_pass = '/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/stem_prep_test.csv'

In [63]:
train_df = pd.read_csv(train_pass)
test_df = pd.read_csv(test_pass)

In [64]:
train_df.head()

,bank,feeds,grades,date,prep_feeds,cups_num,cups_words,stem_text
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1.0,2017-02-16 16:10:00,лет являюсь клиентом этого банка но последний ...,7,отвратительное отношение клиентам ой ужас,лет явля клиент эт банк но последн виз прост о...
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2.0,2016-12-13 01:05:00,ростовнадону ул ленина часов в данном офисе не...,3,г в ао,ростовнадон ул ленин час в дан офис не оказа н...
2,alfabank,Здравствуйте!Ранее уже оставлял отзыв о вашем ...,NaN,2019-06-28 13:54:00,здравствуйте восклицание ранее оставлял отзыв ...,4,в но а фио,здравств восклицан ран оставля отз ваш банк не...
3,vtb,Обращаюсь к Вам с жалобой на незаконное списан...,NaN,2020-07-15 14:54:00,обращаюсь вам жалобой на незаконное списание д...,30,пао втб уфк пао втб пао втб n фз ...,обраща вам жалоб на незакон списан денежн сред...
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2.0,2020-04-08 06:38:00,имею потребительский кредит взятый в связьбанк...,4,в в псб,им потребительск кред взят в связьбанк перешед...


In [65]:
test_df.head()

,bank,feeds,date,prep_feeds,cups_num,cups_words,stem_text
0,sberbank,Оформляем ипотеку в Сбербанке. 22.06.2020 были...,01.07.2020 10:53,оформляем ипотеку в сбербанке подгружены необх...,0,NaN,оформля ипотек в сбербанк подгруж необходим до...
1,alfabank,Краткое содержание: не рекомендую брать кредит...,20.06.2019 13:19,краткое содержание не рекомендую брать кредит ...,3,в в в,кратк содержан не рекоменд брат кред в эт банк...
2,v-express-bank,"Добрый день, уважаемые сотрудники службы контр...",20.02.2016 11:46,добрый день уважаемые сотрудники службы контро...,3,м цб,добр ден уважа сотрудник служб контрол качеств...
3,homecreditbank,"Обращался за получением карты ""Зеленая польза""...",06.05.2019 15:48,обращался получением карты зеленая польза сотр...,0,NaN,обраща получен карт зелен польз сотрудник сооб...
4,vtb,20.05.2016 обратилась в отделение банка на про...,23.05.2016 15:41,обратилась в отделение банка на проспекте лени...,9,втб втб втб бесплатно колоссальный боль...,обрат в отделен банк на проспект ленин в отдел...


In [33]:
# Выберем для трейна только те коменты, где есть оценка
TRAIN_1 = train_df[train_df.notna().grades]

In [67]:
TRAIN_2 = train_df[train_df.grades.isna()]

## **TF-IDF**

In [68]:
vect_words = TfidfVectorizer(max_features=81000, analyzer='word', ngram_range=(1, 1))
vect_chars = TfidfVectorizer(max_features=27600, analyzer='char', ngram_range=(1, 3))

In [69]:
all_words_train = vect_words.fit_transform(TRAIN_1.stem_text)
all_chars_train = vect_chars.fit_transform(TRAIN_1.stem_text)

In [70]:
all_words_train.shape

(51476, 65291)

In [71]:
all_chars_train.shape

(51476, 24989)

In [72]:
# Пропуски в train
all_words_train_2 = vect_words.transform(TRAIN_2.stem_text)
all_chars_train_2 = vect_chars.transform(TRAIN_2.stem_text)

In [73]:
all_words_train_2.shape

(23524, 65291)

In [38]:
all_words_test = vect_words.transform(test_df.stem_text)
all_chars_test = vect_chars.transform(test_df.stem_text)

In [42]:
all_words_test.shape

(17220, 65291)

In [43]:
all_chars_test.shape

(17220, 24989)

In [74]:
# Соберем матрицу признаков TF-IDF для теста и трейна
train_feats = sparse.hstack([all_words_train, all_chars_train])
train_2_feats = sparse.hstack([all_words_train_2, all_chars_train_2])
test_feats = sparse.hstack([all_words_test, all_chars_test])

In [41]:
train_feats.shape

(51476, 90280)

# Обучение модели

Возьмем LR с 5 фолдами

In [45]:
#model_LR_CV5 = LogisticRegressionCV(solver='saga', cv=5, scoring='f1_score')

In [46]:
#model_LR_CV5.fit(train_feats, TRAIN_1.grades.values)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

LogisticRegressionCV(cv=5, solver='saga')

## **Валидация**

In [ ]:
#scores = cross_validate(model_LR_CV5, train_feats, TRAIN_1.grades.values, cv=5,
#                         scoring=('f1_score'),
#                         return_train_score=True)

#np.mean(scores['test_score'])

In [48]:
predicted = model_LR_CV5.predict(train_feats)

In [51]:
f1_score(predicted, TRAIN_1.grades.values, average='micro')

0.7976921283705027

In [53]:
# Сохраним модельку
with open('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/model_tfidf_train_1.pickle', 'wb') as f:
    pickle.dump(model_LR_CV5, f)

In [75]:
with open('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/model_tfidf_train_1.pickle', 'rb') as f:
    model_LR_CV5 = pickle.load(f)

In [76]:
#predicted = model_LR_CV5.predict(train_feats)

In [77]:
#f1_score(predicted, TRAIN_1.grades.values, average='micro')

0.7976921283705027

In [54]:
# Сделаем прогноз
test_predicted = model_LR_CV5.predict(test_feats)

In [58]:
test_predicted = test_predicted.astype(int)

In [59]:
sol = pd.DataFrame({'inds': test_df.index,
                    'grades': test_predicted})
sol

,inds,grades
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
17215,17215,1
17216,17216,1
17217,17217,1
17218,17218,1


In [8]:
sol.to_csv('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/solution_1.csv', index=False)

NameError: name 'sol' is not defined

# Используем пропуски

In [80]:
train_1_predicted_prob = model_LR_CV5.predict_proba(train_feats)

In [82]:
train_2_predicted = model_LR_CV5.predict(train_2_feats)

In [89]:
train_2_predicted_prob = model_LR_CV5.predict_proba(train_2_feats)

In [93]:
train_2_predicted_prob.max(axis=1)

array([0.78389362, 0.93540707, 0.94422073, ..., 0.85460029, 0.50363238,
       0.89184511])

In [94]:
# Добавим колонку с вероятностями класса
TRAIN_2['proba'] = train_2_predicted_prob.max(axis=1)

<ipython-input-94-a24edb145c6f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAIN_2['proba'] = train_2_predicted_prob.max(axis=1)


In [84]:
train_2_predicted = train_2_predicted.astype(int)

In [95]:
# Добавим классы
TRAIN_2['grades'] = train_2_predicted

<ipython-input-95-245555986b22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TRAIN_2['grades'] = train_2_predicted


In [111]:
# Отфильтруем неуверенные прогнозы
# неуверенные прогнозы на всех классах кроме 1
UNSURE_tr_2 = TRAIN_2[(TRAIN_2.proba < 0.8) & (TRAIN_2.grades != 1)]

In [120]:
UNSURE_tr_2.grades.value_counts()

5    1157
2     138
3      31
4      13
Name: grades, dtype: int64

In [117]:
# Неуверенные прогнозы на 1 классе
UNSURE_tr_2_first_class = TRAIN_2[(TRAIN_2.proba < 0.45) & (TRAIN_2.grades == 1)]

In [121]:
UNSURE_tr_2_first_class.grades.value_counts()

1    1156
Name: grades, dtype: int64

In [115]:
# Сохраним эти прогнозы для разметки
UNSURE_tr_2.to_excel('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/UNSURE_tr_2_wt_first.xlsx', index=False)

In [118]:
UNSURE_tr_2_first_class.to_excel('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/UNSURE_tr_2_first_class.xlsx', index=False)

In [123]:
TRAIN_2.to_csv('/content/drive/MyDrive/Kaggle/HSE_NLP_Bootcamp_Sentiment_Analysis/TRAIN_2_w_pred.csv', index=False)